# MNIST 3.1 - Convolutional 

This document serves as my notes on the Deep Learning Without a PhD seminar by Google. 

* [Video](https://www.youtube.com/watch?v=vq2nnJ4g6N0)
* [Slides - 1](https://docs.google.com/presentation/d/1TVixw6ItiZ8igjp6U17tcgoFrLSaHWQmMOwjlgQY9co/pub?slide=id.p)
* [Slides - 2](https://docs.google.com/presentation/d/e/2PACX-1vRouwj_3cYsmLrNNI3Uq5gv5-hYp_QFdeoan2GlxKgIZRSejozruAbVV0IMXBoPsINB7Jw92vJo2EAM/pub?slide=id.p)

This notebook covers the example of trying to classify MNIST digits with:
* a **bigger** convolutional network than in 3.0, 
* as well as using **bigger** filters, giving us more degrees of freedom
* dropout on the fully connected layer

**Note:** We dont have dropout on the conv layers, because we are trying to add degrees of freedom there.

# Setup

## Imports

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data as mnist_data
import math
import pickle

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

## Load Data

In [2]:
# Download images and labels into mnist.test (10K images+labels) and mnist.train (60K images+labels)
mnist = mnist_data.read_data_sets("data", one_hot=True, reshape=False, validation_size=0)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


# Our Model

In this notebook, we will code up the following model:

![](images/conv-model-2.png)

**Note:** The output channels of one layer, matches the input channels of the next layer.

For clarity, we can define all our sizes upfront: 

In [3]:
batchSize = None;       # put None for right now as we dont know yet

imgHeight = 28
imgWidth = 28
numOfColors = 1         # gray scale images

numberOfClasses = 10    # 10 classes: 0-9

Now we define the size of our convolutional layers. **Note**: This corresponds to **number of different filters** we are going to use on this layer. Visually, this is the number of output/slices we produce above.

In [4]:
# Size of each layer:
sizeLayerOne = 6
sizeLayerTwo = 12
sizeLayerThree = 24

Now we define the sizes of our fully connected layer, and the output layer as before: 

In [5]:
sizeLayerFour = 200
sizeLayerFive = numberOfClasses       # the final layer is the output layer

Now we will define the stride size, according to the image above. According to [this stackoverflow question](https://stackoverflow.com/questions/34642595/tensorflow-strides-argument): 

The first 1 is the batch: You don't usually want to skip over examples in your batch, or you shouldn't have included them in the first place. :)

The last 1 is the depth of the convolution: You don't usually want to skip inputs, for the same reason.

The conv2d operator is more general, so you could create convolutions that slide the window along other dimensions, but that's not a typical use in convnets. The typical use is to use them spatially.

In [6]:
stridesL1 = [1,1,1,1]
stridesL2 = [1,2,2,1]
stridesL3 = [1,2,2,1]

As before, lets begin with our inputs and placeholders:

In [7]:
X_img = tf.placeholder(tf.float32, [batchSize, imgHeight, imgWidth, numOfColors], name="X_img")
Y_True = tf.placeholder(tf.float32, [batchSize, 10])
learningRate = tf.placeholder(tf.float32)
probKeep = tf.placeholder(tf.float32, name="Prob_Keep")

With a global variable holding our dropout prob:

In [8]:
PROBKEEP = 0.75

Now we can actually define our layers:

In [9]:
with tf.name_scope("Layer_1_Conv"):
    W1 = tf.Variable(tf.truncated_normal([6,6,1,sizeLayerOne], stddev=0.1), name="Weights")
    
    b1 = tf.Variable(tf.ones([sizeLayerOne])/10, name="Bias")
    
    Y1 = tf.nn.relu(tf.nn.conv2d(X_img, W1, strides=stridesL1, padding='SAME') + b1)

In [10]:
with tf.name_scope("Layer_2_Conv"):
    W2 = tf.Variable(tf.truncated_normal([5,5,sizeLayerOne,sizeLayerTwo], stddev=0.1), name="Weights")
    
    b2 = tf.Variable(tf.ones([sizeLayerTwo])/10, name="Bias")
    
    Y2 = tf.nn.relu(tf.nn.conv2d(Y1, W2, strides=stridesL2, padding='SAME') + b2)

In [11]:
with tf.name_scope("Layer_3_Conv"):
    W3 = tf.Variable(tf.truncated_normal([4,4,sizeLayerTwo,sizeLayerThree], stddev=0.1), name="Weights")
    
    b3 = tf.Variable(tf.ones([sizeLayerThree])/10, name="Bias")
    
    Y3 = tf.nn.relu(tf.nn.conv2d(Y2, W3, strides=stridesL3, padding='SAME') + b3)

**Note:** We need to be careful about sizes - as we can see in the image above, the result of this layer will be a $7 \times 7 \times 12$ tensor. We want to flatten this, so we need a fully connected layer with a weight matrix of size: $7 \times 7 \times \texttt{sizeLayerThree}$

In [12]:
with tf.name_scope("Layer_4_FC"):
    W4 = tf.Variable(tf.truncated_normal([7*7*sizeLayerThree,sizeLayerFour], stddev=0.1), name="Weights")
    
    b4 = tf.Variable(tf.ones([sizeLayerFour])/10, name="Bias")
    
    # flatten the last layers output
    Y3_Flat = tf.reshape(Y3, shape=[-1, 7 * 7 * sizeLayerThree], name="flatten")
    
    Y4 = tf.nn.relu(tf.matmul(Y3_Flat, W4) + b4)
    
    # Dropout
    Y4_Dropout = tf.nn.dropout(Y4, probKeep)

In [13]:
with tf.name_scope("Output_Layer"):
    W5 = tf.Variable(tf.truncated_normal([sizeLayerFour, sizeLayerFive], stddev=0.1), name="Weights")
    
    b5 = tf.Variable(tf.ones([sizeLayerFive])/10, name="Bias")
    
    Y_logits = tf.matmul(Y4_Dropout, W5) + b5
    
    Y_Pred = tf.nn.softmax(Y_logits, name="Activation")

# Training

As before:

In [14]:
# normalized cross entropy loss
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=Y_logits, labels=Y_True)
cross_entropy = tf.reduce_mean(cross_entropy)*100

# accuracy of the trained model, between 0 (worst) and 1 (best)
correct_prediction = tf.equal(tf.argmax(Y_Pred, 1), tf.argmax(Y_True, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# training step
trainStep = tf.train.AdamOptimizer(learningRate).minimize(cross_entropy)

In [15]:
# training parameters
numberOfBatches = 10000
batchSize = 100

trainingAccuracyList = []
trainingLossList = []
testAccuracyList = []
testLossList = []

# learning rate decay
maxLearningRate = 0.003
minLearningRate = 0.0001
decaySpeed = 2000.0   # 0.003-0.0001-2000=>0.9826 done in 5000 iterations

In [18]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    
    # actually initialize our variables
    sess.run(init)
    
    # batch-minimization loop
    for i in range(numberOfBatches):
        # get this batches data
        batch_X, batch_Y = mnist.train.next_batch(batchSize)
        
        # calculate new learning rate for this batch:
        learning_Rate = minLearningRate + (maxLearningRate - minLearningRate) * math.exp(-i/decaySpeed)
    
        # setup this batches input dictionary
        #     - including the new learning rate
        #     - and the dropout/probKeep value
        train_data = {X_img: batch_X, Y_True: batch_Y, learningRate: learning_Rate, probKeep: PROBKEEP}
        
        # run the training step on this batch
        sess.run(trainStep, feed_dict=train_data)
        
        # check our accuracy on training and test data 
        # while resetting dropout! 
        if i%100 == 0:
            # compute our success on the training data
            train_data_to_test_on = {X_img: batch_X, Y_True: batch_Y, learningRate: learning_Rate, probKeep: 1.0}
            trainAcc, trainLoss = sess.run([accuracy, cross_entropy], feed_dict=train_data_to_test_on)
    
            # compute our success on the test data
            test_data = {X_img: mnist.test.images, Y_True: mnist.test.labels, probKeep: 1.0}
            testAcc,testLoss = sess.run([accuracy, cross_entropy], feed_dict=test_data)
            # print("Train " + str(i) + ": accuracy:" + str(trainAcc) + " loss: " + str(trainLoss))
            # print("Test " + str(i) + ": accuracy:" + str(testAcc) + " loss: " + str(testLoss))
            
            trainingAccuracyList.append(trainAcc)
            trainingLossList.append(trainLoss)
            testAccuracyList.append(testAcc)
            testLossList.append(testLoss)
            
            print("Batch number: ",i, "lr: ", learning_Rate, "Test Loss: ", testLoss)

Batch number:  0 lr:  0.003 Test Loss:  273.419
Batch number:  100 lr:  0.0028585653310520707 Test Loss:  16.0462
Batch number:  200 lr:  0.0027240285123042826 Test Loss:  9.62034
Batch number:  300 lr:  0.0025960531316326675 Test Loss:  8.41056
Batch number:  400 lr:  0.0024743191839261473 Test Loss:  6.36464
Batch number:  500 lr:  0.002358522270907074 Test Loss:  5.3494
Batch number:  600 lr:  0.002248372839976982 Test Loss:  5.20454
Batch number:  700 lr:  0.002143595460184269 Test Loss:  4.70025
Batch number:  800 lr:  0.002043928133503354 Test Loss:  4.47475
Batch number:  900 lr:  0.001949121639703143 Test Loss:  4.7859
Batch number:  1000 lr:  0.0018589389131666372 Test Loss:  4.09029
Batch number:  1100 lr:  0.0017731544501034114 Test Loss:  3.66008
Batch number:  1200 lr:  0.001691553744672677 Test Loss:  4.0337
Batch number:  1300 lr:  0.0016139327526069466 Test Loss:  3.13788
Batch number:  1400 lr:  0.0015400973809950877 Test Loss:  3.33295
Batch number:  1500 lr:  0.00146

# Plotting

In [ ]:
plt.figure(figsize=(20,8))

# Plot Accuracy
plt.subplot(1,2,1);
plt.plot(trainingAccuracyList, label="Train Acc");
plt.plot(testAccuracyList, label="Test Acc");
plt.title("Accuracy");
plt.legend();

# Plot Loss
plt.subplot(1,2,2);
plt.plot(trainingLossList, label="Test Loss");
plt.plot(testLossList, label="Test Loss");
plt.title("Loss");
plt.legend();

In [ ]:
tailLength = -90

plt.figure(figsize=(20,8))

# Plot Accuracy
plt.subplot(1,2,1);
plt.plot(trainingAccuracyList[tailLength:], label="Train Acc");
plt.plot(testAccuracyList[tailLength:], label="Test Acc");
plt.title("Accuracy");
plt.legend();

# Plot Loss
plt.subplot(1,2,2);
plt.plot(trainingLossList[tailLength:], label="Test Loss");
plt.plot(testLossList[tailLength:], label="Test Loss");
plt.title("Loss");
plt.legend();

# Save Results

In [ ]:
resultsDic = {"trainAcc": trainingAccuracyList, "trainLoss": trainingLossList, "testAcc": testAccuracyList, "testLoss": testLossList}

with open("results/mnist-3.1-results.txt", "wb") as fp:   #Pickling
    pickle.dump(resultsDic, fp)

# Compare with Previous

In [ ]:
with open("results/mnist-3.0-results.txt", "rb") as rp:
    prev_resultsDic = pickle.load(rp)

In [ ]:
tailLength = -90

plt.figure(figsize=(20,8))

# Plot Accuracy
plt.subplot(1,2,1);
plt.plot(trainingAccuracyList[tailLength:], label="Train Acc");
plt.plot(testAccuracyList[tailLength:], label="Test Acc");

plt.plot(prev_resultsDic["trainAcc"][tailLength:], label="2.2 - Train Acc", alpha=0.3, linestyle=':');
plt.plot(prev_resultsDic["testAcc"][tailLength:], label="2.2 - Test Acc", alpha=0.3, linestyle=':');

plt.title("Comparing Amnisccuracy");
plt.legend();

# Plot Loss
plt.subplot(1,2,2);
plt.plot(trainingLossList[tailLength:], label="Test Loss");
plt.plot(testLossList[tailLength:], label="Test Loss", linewidth=5);

plt.plot(prev_resultsDic["trainLoss"][tailLength:], label="2.2 - Train Loss", alpha=0.3, linestyle=':');
plt.plot(prev_resultsDic["testLoss"][tailLength:], label="2.2 - Test Loss", alpha=0.3, linestyle=':');

plt.title("Loss");
plt.legend();

So we are now almost at 99% accuracy but not quite! Notice we once again have the rise in our test error! 